# 📄 テキストチャンク化教材：RAG のための事前処理と設計

この教材では、RAG（Retrieval-Augmented Generation）構成の事前処理として不可欠な「チャンク化（text chunking）」について、背景知識から Python での実装例までを段階的に学びます。Weaviate や LangChain などと組み合わせる前提の教材です。

## ✅ ステップ ①：チャンク化とは？

### 📌 概要

- チャンク（chunk）＝検索・埋め込み用の**適切な長さの分割テキスト**
- 多くの LLM はトークン数制限があるため、**長文は分割して扱う必要**がある
- 例：PDF・HTML・Markdown・議事録などの文書を 300〜500 文字ごとに分割

### 📌 なぜ重要か？

- 細かすぎる → コンテキストが失われる
- 長すぎる → 入力制限・ノイズが増える

## ✅ ステップ ②：分割の方法と戦略

### 📌 固定文字数で分割（シンプル）

In [1]:
from textwrap import wrap

text = "...長いテキスト..."
chunks = wrap(text, width=300)
print(chunks)

['...長いテキスト...']


### 📌 センテンス単位 + トークン考慮（NLTK など）

#### 英語など欧文

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(text)
chunks = []
buffer = ""
for sentence in sentences:
    if len(buffer + sentence) < 400:
        buffer += sentence + " "
    else:
        chunks.append(buffer.strip())
        buffer = sentence + " "
if buffer:
    chunks.append(buffer.strip())

#### 日本語（NLTK は不向き、代替として`janome`や`fugashi`などを使用）

In [ ]:
from janome.tokenizer import Tokenizer

text = "これは日本語の文です。文を分割して処理します。"
t = Tokenizer()
sentences = text.split("。")

chunks = []
buffer = ""
for sentence in sentences:
    sentence = sentence.strip()
    if not sentence:
        continue
    if len(buffer + sentence) < 400:
        buffer += sentence + "。"
    else:
        chunks.append(buffer.strip())
        buffer = sentence + "。"
if buffer:
    chunks.append(buffer.strip())

### 📌 LangChain の`RecursiveCharacterTextSplitter`

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", "。", "、", " "]
)
chunks = splitter.split_text(text)

## ✅ ステップ ③：正規表現を用いた前処理・整形

### 📌 ノイズ除去や整形の例

In [ ]:
import re

# 改行・空白の正規化
text = re.sub(r"\s+", " ", text)

# 特定の記号を除去
text = re.sub(r"[【】『』◇◆■□●○▲▼☆★▶︎→→]", "", text)

# セクション見出しで分割（例："第1章"）
chunks = re.split(r"第\d+章", text)

## ✅ ステップ ④：ファイルごとのチャンク化

### 📌 PDF の場合（PyMuPDF）

In [ ]:
import fitz  # PyMuPDF

doc = fitz.open("document.pdf")
text = "\n".join([page.get_text() for page in doc])

### 📌 Markdown/HTML/Text

In [ ]:
with open("file.md", encoding="utf-8") as f:
    text = f.read()

## ✅ ステップ ⑤：チャンクにメタデータを付加

### 📌 ファイル名・ページ番号などを保持

In [ ]:
chunk_records = []
for i, chunk in enumerate(chunks):
    chunk_records.append({
        "content": chunk,
        "source": "document.pdf",
        "page": i + 1
    })

## ✅ ステップ ⑥：Weaviate への登録

### 📌 チャンクとメタデータを Weaviate に保存

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8080")

# スキーマが未作成の場合は事前に作成する
class_obj = {
    "class": "DocumentChunk",
    "properties": [
        {"name": "content", "dataType": ["text"]},
        {"name": "source", "dataType": ["text"]},
        {"name": "page", "dataType": ["int"]}
    ],
    "vectorizer": "text2vec-openai"
}
client.schema.create_class(class_obj)

# チャンクを登録
for record in chunk_records:
    client.data_object.create(record, class_name="DocumentChunk")

## ✅ 実践課題

### 📁 課題 1：任意のテキストファイルを 300 文字ごとに分割し、チャンク一覧を出力

### 📁 課題 2：Markdown 文書から文単位チャンクを抽出し、章ごとのチャンク一覧を作成

### 📁 課題 3：チャンクごとにメタデータ（セクション名・出典など）を保持し、CSV または JSON に出力

### 🚀 最終ゴール

- LLM やベクトル DB と組み合わせる前提として、**扱いやすく検索に適したテキストチャンクを自動生成**できるようになること。